Скачиваем необходимые файлы.

In [1]:
# Скачиваем биопитон.
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 5.2 MB/s 


In [2]:
# Добавляем файлы из репозитория.
!wget https://raw.githubusercontent.com/dreamer1978/HW2_annotation/main/data/scaffolds.hits_from_SwissProt.txt
!wget https://raw.githubusercontent.com/dreamer1978/HW2_annotation/main/data/scaffolds.hits_from_MIL_1.txt
!wget https://raw.githubusercontent.com/dreamer1978/HW2_annotation/main/data/proteins.fasta
!wget https://raw.githubusercontent.com/dreamer1978/HW2_annotation/main/data/gms2.lst

y
--2022-10-23 19:23:23--  https://raw.githubusercontent.com/dreamer1978/HW2_annotation/main/data/scaffolds.hits_from_SwissProt.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66249 (65K) [text/plain]
Saving to: ‘scaffolds.hits_from_SwissProt.txt’

scaffolds.hits_from 100%[===================>]  64.70K  --.-KB/s    in 0.01s   

2022-10-23 19:23:23 (5.56 MB/s) - ‘scaffolds.hits_from_SwissProt.txt’ saved [66249/66249]

--2022-10-23 19:23:23--  https://raw.githubusercontent.com/dreamer1978/HW2_annotation/main/data/scaffolds.hits_from_MIL_1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... con

In [3]:
# Получаем скаффолд из первого задания.
!wget https://raw.githubusercontent.com/dreamer1978/hse22_hw1/main/data/GAP_CLOSE_gapClosed.fa

--2022-10-23 19:23:32--  https://raw.githubusercontent.com/dreamer1978/hse22_hw1/main/data/GAP_CLOSE_gapClosed.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3965608 (3.8M) [text/plain]
Saving to: ‘GAP_CLOSE_gapClosed.fa’

GAP_CLOSE_gapClosed 100%[===================>]   3.78M  --.-KB/s    in 0.07s   

2022-10-23 19:23:33 (54.4 MB/s) - ‘GAP_CLOSE_gapClosed.fa’ saved [3965608/3965608]



In [4]:
# Скачаем файл близкородсивенной бактерии в формате genbank
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=\${PATH}:/root/edirect" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
y
OK, done.

To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [5]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

In [21]:
from Bio import SeqIO

# Переведем скаченный файл из пердыдущего ДЗ в массив типов Bio.SeqRecord.SeqRecord.
scaffolds = {}
for scaffold in SeqIO.parse('GAP_CLOSE_gapClosed.fa', format='fasta'):
  scaffolds[scaffold.id] = scaffold
  scaffold.annotations['molecule_type'] = 'DNA'
# Теперь у нас есть все скаффолды в словаре.

In [22]:
from Bio.SeqFeature import SeqFeature, FeatureLocation

genes = {}
# Добавляем фичи в массивы генов.
# Добавляем координаты для генов.
for gene in SeqIO.parse('proteins.fasta', format='fasta'):
  arr = gene.description.split(' ')

  scaf_name = arr[1]
  start = int(arr[2])
  end = int(arr[3])

  feature = SeqFeature(FeatureLocation(start, end, strand=(arr[4]=='+')), type="CDS")
  feature.qualifiers['locus_tag'] = [arr[0]]
  feature.qualifiers['translation'] = [gene.seq]

  scaffolds[scaf_name].features.append(feature)
  genes[arr[0]] = feature

In [23]:
# Добавляем функции белоков близкородсвенной бактерии MIL-1.
qual_genes_mil1 = {}

for gen in SeqIO.parse('/content/T_oleivorans_MIL_1.gbk', format='genbank'):
  for feature in gen.features :
    if 'protein_id' in feature.qualifiers and 'product' in feature.qualifiers:
      qual_genes_mil1[feature.qualifiers['protein_id'][0]] = feature.qualifiers['product'][0]

In [24]:
# Смотрим структуру файла scaffolds.hits_from_MIL_1.txt.
!head scaffolds.hits_from_MIL_1.txt

2	lcl|HF680312.1_prot_CCU72326.1_1877	99.054	317	3	0	3	319	1	317	0.0	644
2	lcl|HF680312.1_prot_CCU72392.1_1943	34.583	240	138	5	85	307	956	1193	2.33e-30	118
2	lcl|HF680312.1_prot_CCU72759.1_2310	27.667	300	188	8	8	302	709	984	1.02e-24	101
2	lcl|HF680312.1_prot_CCU70543.1_94	42.857	126	71	1	79	204	798	922	5.63e-24	99.4
2	lcl|HF680312.1_prot_CCU70690.1_241	26.140	329	205	10	2	303	591	908	6.26e-23	96.7
2	lcl|HF680312.1_prot_CCU74013.1_3564	38.462	130	76	3	91	219	685	811	4.96e-20	87.8
2	lcl|HF680312.1_prot_CCU71475.1_1026	36.522	115	71	2	86	200	555	667	2.19e-16	76.6
2	lcl|HF680312.1_prot_CCU71362.1_913	24.535	269	187	3	52	306	633	899	2.61e-14	70.5
2	lcl|HF680312.1_prot_CCU70724.1_275	30.534	131	81	3	78	205	1983	2106	6.54e-13	66.2
2	lcl|HF680312.1_prot_CCU70719.1_270	26.087	115	83	1	90	204	15	127	9.14e-12	58.5


In [25]:
# Переносим данный в таблице DataFrame.
import pandas as pd
cols = ['qseqid', 'sseqid', 'pident', 'length', 'misMatch', 'gapOpen', 'qstart', 'qend', 'sstart', 'send', 'e-value', 'bitScore']
df = pd.read_csv('scaffolds.hits_from_MIL_1.txt', sep='\t', header=None, names=cols)

In [26]:
df.head()

,qseqid,sseqid,pident,length,misMatch,gapOpen,qstart,qend,sstart,send,e-value,bitScore
0,2,lcl|HF680312.1_prot_CCU72326.1_1877,99.054,317,3,0,3,319,1,317,0.000000e+00,644.0
1,2,lcl|HF680312.1_prot_CCU72392.1_1943,34.583,240,138,5,85,307,956,1193,2.330000e-30,118.0
2,2,lcl|HF680312.1_prot_CCU72759.1_2310,27.667,300,188,8,8,302,709,984,1.020000e-24,101.0
3,2,lcl|HF680312.1_prot_CCU70543.1_94,42.857,126,71,1,79,204,798,922,5.630000e-24,99.4
4,2,lcl|HF680312.1_prot_CCU70690.1_241,26.140,329,205,10,2,303,591,908,6.260000e-23,96.7


In [27]:
# Сортировка и фильтрация данных.
df = df.sort_values(by='bitScore', ascending=0).drop_duplicates('qseqid')
df = df[df['sseqid'].str.contains("CCU")]
df.head()

,qseqid,sseqid,pident,length,misMatch,gapOpen,qstart,qend,sstart,send,e-value,bitScore
7198,2515,lcl|HF680312.1_prot_CCU73896.1_3447,98.979,2253,23,0,1,2253,1,2253,0.0,4296.0
5770,2053,lcl|HF680312.1_prot_CCU70724.1_275,97.870,2113,43,1,7,2119,1,2111,0.0,4158.0
891,322,lcl|HF680312.1_prot_CCU71900.1_1451,99.878,1640,2,0,1,1640,1,1640,0.0,3406.0
7424,2591,lcl|HF680312.1_prot_CCU73861.1_3412,99.214,1654,13,0,1,1654,1,1654,0.0,3372.0
115,46,lcl|HF680312.1_prot_CCU71621.1_1172,99.690,1611,5,0,1,1611,1,1611,0.0,3338.0


In [30]:
print('Cколько из них удалось аннотировать с помощью сравнения с бактерией MIL-1 :', df.shape[0])

Cколько из них удалось аннотировать с помощью сравнения с бактерией MIL-1 : 3329


In [18]:
# Скачиваем функций белков SwissProt из файла.
!wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
!gzip -d uniprot_sprot.dat.gz
!grep '^ID\|^DE   RecName: Full=' uniprot_sprot.dat > SwissProt_names.txt

--2022-10-23 19:32:09--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 642093634 (612M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 612.35M  21.0MB/s    in 23s     

2022-10-23 19:32:32 (26.3 MB/s) - ‘uniprot_sprot.dat.gz’ saved [642093634/642093634]



In [28]:
for i, row in df.iterrows():
  gene = genes[str(row['qseqid'])]
  match = row['sseqid'].split('_')[2]
  gene.qualifiers['product'] = [qual_genes_mil1[match]]

# Аннотируем геном.
SeqIO.write(scaffolds.values(), 'GENOME.gbk', 'genbank')

71